In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Fonction : Conversion Excel -> datetime
def excel_vers_datetime(date_excel):
    if pd.isna(date_excel):
        return pd.NaT
    try:
        return pd.to_datetime('1900-01-01') + pd.to_timedelta(date_excel - 2, unit='D')
    except:
        return pd.NaT

# Fonction : Nettoyage et préparation des données
def charger_et_nettoyer_donnees(fichier):
    donnees = pd.read_excel(fichier)

    # Détection des colonnes d'horodatage
    colonnes_horodatage = [
        col for col in donnees.columns 
        if donnees[col].dtype == 'float64' and str(donnees[col].iloc[0]).startswith('45791')
    ]
    if not colonnes_horodatage:
        raise ValueError("Aucune colonne d'horodatage détectée.")

    # Conversion des dates
    for col in colonnes_horodatage:
        donnees[col] = donnees[col].apply(excel_vers_datetime)
    donnees['Horodatage_Unifié'] = donnees[colonnes_horodatage[0]]
    donnees.drop(columns=colonnes_horodatage, inplace=True)

    # Colonnes utiles
    colonnes_utiles = [
        'Débit d\'entrée d\'acide m3/h',
        'Débit de vapeur Kg/h',
        'Température de sortie évaporateur en C°',
        'Vide bouilleur en torr',
        'Densité de sortie',
        'Horodatage_Unifié'
    ]
    donnees = donnees[[col for col in donnees.columns if col in colonnes_utiles]]

    # Interpolation des valeurs manquantes
    donnees = donnees.interpolate(method='linear', limit_direction='both')

    # Suppression des débits nuls/négatifs
    donnees = donnees[
        (donnees["Débit d'entrée d'acide m3/h"] > 0) &
        (donnees["Débit de vapeur Kg/h"] > 0)
    ]

    # Détection des outliers (basé sur l'IQR pour Densité_Sortie)
    Q1 = donnees['Densité de sortie'].quantile(0.25)
    Q3 = donnees['Densité de sortie'].quantile(0.75)
    IQR = Q3 - Q1
    donnees = donnees[
        (donnees['Densité de sortie'] >= Q1 - 1.5 * IQR) &
        (donnees['Densité de sortie'] <= Q3 + 1.5 * IQR)
    ]

    # Format datetime
    donnees['Horodatage_Unifié'] = donnees['Horodatage_Unifié'].dt.strftime('%m/%d/%Y %H:%M')

    # Renommage
    donnees.columns = [
        'Débit_Acide_m3h',
        'Débit_Vapeur_kgh',
        'Température_Évaporateur_C',
        'Vide_Bouilleur_torr',
        'Densité_Sortie',
        'Horodatage_Unifié'
    ]
    donnees = donnees[
        ['Horodatage_Unifié', 'Débit_Acide_m3h', 'Débit_Vapeur_kgh',
         'Température_Évaporateur_C', 'Vide_Bouilleur_torr', 'Densité_Sortie']
    ]

    # Sauvegarde
    donnees.to_excel("donnees_nettoyees.xlsx", index=False)
    
    # Afficher les corrélations
    print("📊 Corrélations avec Densité_Sortie :")
    print(donnees.corr(numeric_only=True)['Densité_Sortie'])
    
    return donnees

# Fonction : Entraînement du modèle Ridge Polynomial avec transformations
def entrainer_meilleur_modele(donnees):
    donnees['Horodatage_Unifié'] = pd.to_datetime(donnees['Horodatage_Unifié'], format='%m/%d/%Y %H:%M')
    X = donnees[['Débit_Acide_m3h', 'Débit_Vapeur_kgh', 'Température_Évaporateur_C', 'Vide_Bouilleur_torr']]
    y = donnees['Densité_Sortie']
    
    # Centrer la cible (soustraire la moyenne)
    y_mean = y.mean()
    y_centered = y - y_mean
    
    X_train, X_test, y_train, y_test = train_test_split(X, y_centered, test_size=0.2, random_state=42)

    # Pipeline avec transformation logarithmique pour Vide_Bouilleur_torr
    def log_transform(X):
        X_transformed = X.copy()
        X_transformed['Vide_Bouilleur_torr'] = np.log1p(X_transformed['Vide_Bouilleur_torr'])
        return X_transformed

    # Pipeline Ridge Polynomial
    pipeline_ridge = Pipeline([
        ('log', FunctionTransformer(log_transform, validate=False)),
        ('poly', PolynomialFeatures(degree=2)),
        ('scaler', StandardScaler()),
        ('ridge', Ridge())
    ])

    # Grille pour Ridge
    param_grid_ridge = {
        'ridge__alpha': [0.01, 0.1, 1, 10, 100, 1000],
        'poly__degree': [1, 2]  # Tester degré 1 et 2
    }
    grid_search_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search_ridge.fit(X_train, y_train)

    # Pipeline Random Forest
    pipeline_rf = Pipeline([
        ('scaler', StandardScaler()),
        ('rf', RandomForestRegressor(random_state=42))
    ])

    # Grille pour Random Forest
    param_grid_rf = {
        'rf__n_estimators': [50, 100],
        'rf__max_depth': [None, 10]
    }
    grid_search_rf = GridSearchCV(pipeline_rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search_rf.fit(X_train, y_train)

    # Sélection du meilleur modèle
    if -grid_search_ridge.best_score_ < -grid_search_rf.best_score_:
        best_model = grid_search_ridge.best_estimator_
        print("🔎 Meilleur modèle : Ridge Polynomial")
        print("🔎 Meilleur alpha :", grid_search_ridge.best_params_['ridge__alpha'])
        print("🔎 Meilleur degré :", grid_search_ridge.best_params_['poly__degree'])
    else:
        best_model = grid_search_rf.best_estimator_
        print("🔎 Meilleur modèle : Random Forest")
        print("🔎 Meilleurs paramètres RF :", grid_search_rf.best_params_)

    # Validation croisée imbriquée
    cv_scores = cross_val_score(best_model, X, y_centered, cv=5, scoring='neg_mean_squared_error')
    mean_cv_mse = -cv_scores.mean()
    print("📈 MSE moyenne (CV imbriquée) :", mean_cv_mse)

    # Évaluation sur le test
    y_pred = best_model.predict(X_test)
    y_pred_uncentered = y_pred + y_mean  # Revenir à l'échelle originale
    y_test_uncentered = y_test + y_mean
    mse_test = mean_squared_error(y_test_uncentered, y_pred_uncentered)
    r2_test = r2_score(y_test_uncentered, y_pred_uncentered)
    print("📊 Test R2 :", r2_test)
    print("📉 Test MSE :", mse_test)

    # Sauvegarde du modèle
    joblib.dump(best_model, 'modele_final_ridge_poly.pkl')
    print("✅ Modèle sauvegardé sous 'modele_final_ridge_poly.pkl'")
    return best_model, y_mean

# Fonction : Prédiction sur nouvelles données
def predire_nouvelles_donnees(model, y_mean):
    exemples = pd.DataFrame({
        'Débit_Acide_m3h': [30.0, 29.9, 29.8],
        'Débit_Vapeur_kgh': [3525, 3550, 3580],
        'Température_Évaporateur_C': [92.36, 92.35, 92.34],
        'Vide_Bouilleur_torr': [59.1, 59.0, 58.9]
    })
    predictions = model.predict(exemples) + y_mean  # Revenir à l'échelle originale
    print("🔮 Prédictions sur nouveaux exemples :")
    print(predictions)
    return predictions

# Exécution principale
if __name__ == "__main__":
    fichier = 'DATA-CONC1-140525-050625.xlsx'
    donnees = charger_et_nettoyer_donnees(fichier)
    modele, y_mean = entrainer_meilleur_modele(donnees)
    predire_nouvelles_donnees(modele, y_mean)